# Create a Arbitrage Betting Strategy

#### Install PolyMarket API library

In [1]:
!pip install py-clob-client

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for eth-account>=0.13.0 from https://files.pythonhosted.org/packages/46/18/088fb250018cbe665bc2111974301b2d59f294a565aff7564c4df6878da2/eth_account-0.13.7-py3-none-any.whl.metadata
  Obtaining dependency information for eth-utils>=4.1.1 from https://files.pythonhosted.org/packages/c4/c6/0417a92e6a3fc9b85f5a8380d9f9d43b69ba836a90e45f79f9ae74d41e53/eth_utils-5.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for poly_eip712_structs>=0.0.1 from https://files.pythonhosted.org/packages/5a/d7/ff1cfba1c3a3d5d6851d7bef5e4ad19710ed6d03e149dc183111d103acab/poly_eip712_structs-0.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for py-order-utils>=0.3.2 from https://files.pythonhosted.org/packages/29/68/b0a971b064b3236fce7307bd5c180409cccd9b207ec459274bdb4e401ec0/py_order_utils-0.3.2-py3-none-any.whl.metadata
  Obtaining dependency

### PolyMarket API
https://docs.polymarket.com/quickstart/introduction/main

### NYC Mayoral Race 2025

#### Event: NYC Mayoral Election:
https://polymarket.com/event/new-york-city-mayoral-election?tid=1755656631578

#### Markets: 
- Mamdani
- Cuomo
- Adams
- Silwa

#### Get single event data using gamma api

In [24]:
import requests

r = requests.get("https://gamma-api.polymarket.com/events?id=23246")
nyc_mayoral_election_event = r.json()
nyc_mayoral_election_event

[{'id': '23246',
  'ticker': 'new-york-city-mayoral-election',
  'slug': 'new-york-city-mayoral-election',
  'title': 'New York City Mayoral Election',
  'description': 'The 2025 New York City mayoral election will be held on November 4, 2025.\n\nThis market will resolve according to the candidate wins the election.\n\nThe primary resolution source for this market will be a consensus of credible reporting, however if there is any ambiguity in the results this market will resolve according to official information from New York City.\n\n\n',
  'resolutionSource': '',
  'startDate': '2025-04-22T16:14:22.792059Z',
  'creationDate': '2025-04-22T16:14:22.792054Z',
  'endDate': '2025-11-04T12:00:00Z',
  'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/new-york-city-mayoral-election-al8KR4km2AQB.png',
  'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/new-york-city-mayoral-election-al8KR4km2AQB.png',
  'active': True,
  'closed': False,
  'archived': False,
  'new': 

In [26]:
for market in nyc_mayoral_election_event[0]['markets']:
    print(market['id'], market['question'], 'outcomePrices' in market and market['outcomePrices'])
    print('clobTokenIds' in market and market['clobTokenIds'])
    print("----------------------------")

538928 Will Eric Adams win the 2025 NYC mayoral election? ["0.052", "0.948"]
["12541509255877010177934715422358422748052378132651567836811989084820478865881", "557149893577824223387740663828120572195409127574902734254512226734855221907"]
----------------------------
538929 Will Andrew Cuomo win the 2025 NYC mayoral election? ["0.101", "0.899"]
["72685162394098505217895638060393901041260225434938300730127268362092284806692", "104468181147316868388088006861839293041095272602974154655578369735976654024471"]
----------------------------
538930 Will Curtis Sliwa win the 2025 NYC mayoral election? ["0.0065", "0.9935"]
["106645483208866512069057468273735379467092160028743318106715987586591689242591", "42541673615301895829890290486226257940966769125829226067368474110048691276042"]
----------------------------
538931 Will Jim Walden win the 2025 NYC mayoral election? ["0.0045", "0.9955"]
["10142872462975275995759595767341515402450823584364917228661247714894341308969", "3323369066986882543667397

#### Asset ID Mapping

In [56]:
ASSET_ID_MAPPING = {
    "33945469250963963541781051637999677727672635213493648594066577298999471399137": "Mamdani YES",
    "105832362350788616148612362642992403996714020918558917275151746177525518770551": "Mamdani NO",
    "72685162394098505217895638060393901041260225434938300730127268362092284806692": "Cuomo YES",
    "104468181147316868388088006861839293041095272602974154655578369735976654024471": "Cuomo NO"
}

#### Define Order Book

In [51]:
from collections import defaultdict

order_books = defaultdict(lambda: {"bids": {}, "asks": {}})

In [53]:
def update_order_book(asset_id, side, price, size):
    book_side = order_books[asset_id]["bids"] if side == "buy" else order_books[asset_id]["asks"]
    if size == 0:
        book_side.pop(price, None)
    else:
        book_side[price] = size

In [57]:
def print_top_of_book():
    # Print a snapshot (top of book)
    for aid, book in order_books.items():
        top_bid = max(book["bids"].items(), key=lambda x: x[0], default=(None, None))
        top_ask = min(book["asks"].items(), key=lambda x: x[0], default=(None, None))
        print(f"{ASSET_ID_MAPPING[aid]} | Best Bid: {top_bid} | Best Ask: {top_ask}")

In [52]:
def print_top_of_book_single_assest(book):
    if book["bids"]:
        best_bid = max(book["bids"].items())
    else:
        best_bid = (None, None)
    if book["asks"]:
        best_ask = min(book["asks"].items())
    else:
        best_ask = (None, None)
    print(f"Best Bid: {best_bid} | Best Ask: {best_ask}")

#### Get websocket data using Order Book API

In [58]:
import json
import threading
from websocket import WebSocketApp

# WebSocket endpoint for Polymarket CLOB service
WS_URL_BASE = "wss://ws-subscriptions-clob.polymarket.com"

# Your target tokens (clobTokenIds)
ASSET_IDS = [
    "33945469250963963541781051637999677727672635213493648594066577298999471399137",  # Mamdani YES
    "105832362350788616148612362642992403996714020918558917275151746177525518770551",   # Mamdani NO
    "72685162394098505217895638060393901041260225434938300730127268362092284806692",  # Cuomo YES
    "104468181147316868388088006861839293041095272602974154655578369735976654024471"   # Cuomo NO
]

CHANNEL_TYPE = "market"  # use market for public price/book updates

class PolymarketWebSocket:
    def __init__(self, url_base, channel_type, asset_ids):
        self.url = f"{url_base}/ws/{channel_type}"
        self.channel_type = channel_type
        self.asset_ids = asset_ids
        self.ws = WebSocketApp(
            self.url,
            on_open=self.on_open,
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close,
        )

    def on_open(self, ws):
        print(f"Connected to {self.url}")
        # Subscribe to assets
        subscribe_payload = {
            "assets_ids": self.asset_ids,
            "type": self.channel_type
        }
        ws.send(json.dumps(subscribe_payload))
        # Start periodic ping to keep connection alive
        threading.Thread(target=self.ping, args=(ws,), daemon=True).start()

    def on_message(self, ws, message):
        msgs = json.loads(message)

        # Ensure msgs is always a list
        if isinstance(msgs, dict):
            msgs = [msgs]

        for msg in msgs:
            event_type = msg.get("event_type")
            print(f"\nReceived message — event_type: {event_type}")
            print(json.dumps(msg, indent=2))

            if event_type == "book":
                self.handle_book(msg)
            elif event_type == "price_change":
                self.handle_price_change(msg)
            elif event_type == "last_trade_price":
                self.handle_last_trade(msg)
            elif event_type == "tick_size_change":
                self.handle_tick_size_change(msg)
            else:
                print("Unrecognized message:", msg)
            
        print_top_of_book()

    def on_error(self, ws, error):
        print("WebSocket error:", error)

    def on_close(self, ws, close_status_code, close_msg):
        print("WebSocket closed:", close_status_code, close_msg)

    def ping(self, ws):
        import time
        while True:
            ws.send("PING")
            time.sleep(10)

    def run(self):
        self.ws.run_forever()

    # Handler stubs — customize as needed
    def handle_book(self, msg):
        asset_id = msg["asset_id"]
        bids = {float(entry["price"]): float(entry["size"]) for entry in msg["bids"]}
        asks = {float(entry["price"]): float(entry["size"]) for entry in msg["asks"]}
        order_books[asset_id]["bids"] = bids
        order_books[asset_id]["asks"] = asks
        print(f"[BOOK SNAPSHOT] {asset_id}")
        print_top_of_book_single_assest(order_books[asset_id])

    def handle_price_change(self, msg):
        print("Price change update:", msg.get("changes"))
        asset_id = msg["asset_id"]
        for change in msg["changes"]:
            price = float(change["price"])
            size = float(change["size"])
            side = change["side"].lower()
            update_order_book(asset_id, side, price, size)
            
        print(f"[PRICE UPDATE] {asset_id}")
        print_top_of_book_single_assest(order_books[asset_id])

    def handle_last_trade(self, msg):
        print("Last trade price:", msg.get("price"), "size:", msg.get("size"))

    def handle_tick_size_change(self, msg):
        print("Tick size change:", msg.get("old_tick_size"), "→", msg.get("new_tick_size"))

if __name__ == "__main__":
    client = PolymarketWebSocket(WS_URL_BASE, CHANNEL_TYPE, ASSET_IDS)
    client.run()


Connected to wss://ws-subscriptions-clob.polymarket.com/ws/market

Received message — event_type: book
{
  "market": "0xc40cbb2d7f5d2c43c624bd5b1d0b18cd3d0682b3937363ec7c1ad1d13bee107e",
  "asset_id": "72685162394098505217895638060393901041260225434938300730127268362092284806692",
  "timestamp": "1755746812843",
  "hash": "3ea790bf929eb84cac7157a0587fb7eb9d0a7d9d",
  "bids": [
    {
      "price": "0.001",
      "size": "592088"
    },
    {
      "price": "0.002",
      "size": "53088"
    },
    {
      "price": "0.003",
      "size": "18000"
    },
    {
      "price": "0.004",
      "size": "8000"
    },
    {
      "price": "0.005",
      "size": "215"
    },
    {
      "price": "0.015",
      "size": "2697"
    },
    {
      "price": "0.016",
      "size": "5000"
    },
    {
      "price": "0.03",
      "size": "3200"
    },
    {
      "price": "0.031",
      "size": "2000"
    },
    {
      "price": "0.042",
      "size": "200"
    },
    {
      "price": "0.043",
      "si


Received message — event_type: price_change
{
  "asset_id": "104468181147316868388088006861839293041095272602974154655578369735976654024471",
  "changes": [
    {
      "price": "0.766",
      "side": "BUY",
      "size": "0"
    }
  ],
  "event_type": "price_change",
  "hash": "65097fce26e651881b6c2862ce6eb7dacabc3732",
  "market": "0xc40cbb2d7f5d2c43c624bd5b1d0b18cd3d0682b3937363ec7c1ad1d13bee107e",
  "timestamp": "1755746814332"
}
Price change update: [{'price': '0.766', 'side': 'BUY', 'size': '0'}]
[PRICE UPDATE] 104468181147316868388088006861839293041095272602974154655578369735976654024471
Best Bid: (0.9, 145.0) | Best Ask: (0.903, 449.0)
Mamdani YES | Best Bid: (0.837, 3.0) | Best Ask: (0.838, 7648.8)
Mamdani NO | Best Bid: (0.162, 7648.8) | Best Ask: (0.163, 3.0)
Cuomo NO | Best Bid: (0.9, 145.0) | Best Ask: (0.903, 449.0)
Cuomo YES | Best Bid: (0.097, 449.0) | Best Ask: (0.099, 19.44)

Received message — event_type: price_change
{
  "asset_id": "7268516239409850521789563806039


Received message — event_type: price_change
{
  "asset_id": "104468181147316868388088006861839293041095272602974154655578369735976654024471",
  "changes": [
    {
      "price": "0.89",
      "side": "BUY",
      "size": "422"
    }
  ],
  "event_type": "price_change",
  "hash": "0c38ec98cc6304d39abd26e0ca1a43fd393cc4ff",
  "market": "0xc40cbb2d7f5d2c43c624bd5b1d0b18cd3d0682b3937363ec7c1ad1d13bee107e",
  "timestamp": "1755746816694"
}
Price change update: [{'price': '0.89', 'side': 'BUY', 'size': '422'}]
[PRICE UPDATE] 104468181147316868388088006861839293041095272602974154655578369735976654024471
Best Bid: (0.9, 145.0) | Best Ask: (0.903, 449.0)
Mamdani YES | Best Bid: (0.837, 3.0) | Best Ask: (0.838, 7648.8)
Mamdani NO | Best Bid: (0.162, 7648.8) | Best Ask: (0.163, 3.0)
Cuomo NO | Best Bid: (0.9, 145.0) | Best Ask: (0.903, 449.0)
Cuomo YES | Best Bid: (0.097, 449.0) | Best Ask: (0.099, 19.44)

Received message — event_type: price_change
{
  "asset_id": "72685162394098505217895638060


Received message — event_type: price_change
{
  "asset_id": "104468181147316868388088006861839293041095272602974154655578369735976654024471",
  "changes": [
    {
      "price": "0.916",
      "side": "SELL",
      "size": "15311"
    }
  ],
  "event_type": "price_change",
  "hash": "5362113e2f4d7af7d47d25a2ec9ef9d7263d8b6b",
  "market": "0xc40cbb2d7f5d2c43c624bd5b1d0b18cd3d0682b3937363ec7c1ad1d13bee107e",
  "timestamp": "1755746818827"
}
Price change update: [{'price': '0.916', 'side': 'SELL', 'size': '15311'}]
[PRICE UPDATE] 104468181147316868388088006861839293041095272602974154655578369735976654024471
Best Bid: (0.9, 145.0) | Best Ask: (0.903, 449.0)
Mamdani YES | Best Bid: (0.837, 3.0) | Best Ask: (0.838, 7648.8)
Mamdani NO | Best Bid: (0.162, 7648.8) | Best Ask: (0.163, 3.0)
Cuomo NO | Best Bid: (0.9, 145.0) | Best Ask: (0.903, 449.0)
Cuomo YES | Best Bid: (0.097, 449.0) | Best Ask: (0.099, 19.44)

Received message — event_type: price_change
{
  "asset_id": "726851623940985052178


Received message — event_type: price_change
{
  "asset_id": "33945469250963963541781051637999677727672635213493648594066577298999471399137",
  "changes": [
    {
      "price": "0.836",
      "side": "BUY",
      "size": "29884.44"
    }
  ],
  "event_type": "price_change",
  "hash": "fb3cfb965f27c8019d13fc47be63d3ab1786fb89",
  "market": "0xebddfcf7b4401dade8b4031770a1ab942b01854f3bed453d5df9425cd9f211a9",
  "timestamp": "1755746826811"
}
Price change update: [{'price': '0.836', 'side': 'BUY', 'size': '29884.44'}]
[PRICE UPDATE] 33945469250963963541781051637999677727672635213493648594066577298999471399137
Best Bid: (0.837, 3.0) | Best Ask: (0.838, 7648.8)
Mamdani YES | Best Bid: (0.837, 3.0) | Best Ask: (0.838, 7648.8)
Mamdani NO | Best Bid: (0.162, 7648.8) | Best Ask: (0.163, 3.0)
Cuomo NO | Best Bid: (0.9, 145.0) | Best Ask: (0.903, 449.0)
Cuomo YES | Best Bid: (0.097, 449.0) | Best Ask: (0.099, 19.44)

Received message — event_type: price_change
{
  "asset_id": "105832362350788616

...


Received message — event_type: price_change
{
  "asset_id": "33945469250963963541781051637999677727672635213493648594066577298999471399137",
  "changes": [
    {
      "price": "0.824",
      "side": "BUY",
      "size": "13879.66"
    }
  ],
  "event_type": "price_change",
  "hash": "6170ace750714bb81e7a529ba5b244c256337383",
  "market": "0xebddfcf7b4401dade8b4031770a1ab942b01854f3bed453d5df9425cd9f211a9",
  "timestamp": "1755746847043"
}
Price change update: [{'price': '0.824', 'side': 'BUY', 'size': '13879.66'}]
[PRICE UPDATE] 33945469250963963541781051637999677727672635213493648594066577298999471399137
Best Bid: (0.837, 19.0) | Best Ask: (0.838, 7648.8)
Mamdani YES | Best Bid: (0.837, 19.0) | Best Ask: (0.838, 7648.8)
Mamdani NO | Best Bid: (0.162, 7648.8) | Best Ask: (0.163, 19.0)
Cuomo NO | Best Bid: (0.9, 145.0) | Best Ask: (0.903, 449.0)
Cuomo YES | Best Bid: (0.097, 449.0) | Best Ask: (0.099, 19.44)

Received message — event_type: price_change
{
  "asset_id": "105832362350788


Received message — event_type: price_change
{
  "asset_id": "33945469250963963541781051637999677727672635213493648594066577298999471399137",
  "changes": [
    {
      "price": "0.818",
      "side": "BUY",
      "size": "32444"
    }
  ],
  "event_type": "price_change",
  "hash": "c6b460b7fd7f5b1982f065b88542223bf685162d",
  "market": "0xebddfcf7b4401dade8b4031770a1ab942b01854f3bed453d5df9425cd9f211a9",
  "timestamp": "1755746887949"
}
Price change update: [{'price': '0.818', 'side': 'BUY', 'size': '32444'}]
[PRICE UPDATE] 33945469250963963541781051637999677727672635213493648594066577298999471399137
Best Bid: (0.837, 19.0) | Best Ask: (0.838, 7648.8)
Mamdani YES | Best Bid: (0.837, 19.0) | Best Ask: (0.838, 7648.8)
Mamdani NO | Best Bid: (0.162, 7648.8) | Best Ask: (0.163, 19.0)
Cuomo NO | Best Bid: (0.9, 145.0) | Best Ask: (0.903, 449.0)
Cuomo YES | Best Bid: (0.097, 449.0) | Best Ask: (0.099, 19.44)

Received message — event_type: price_change
{
  "asset_id": "105832362350788616148


Received message — event_type: price_change
{
  "asset_id": "104468181147316868388088006861839293041095272602974154655578369735976654024471",
  "changes": [
    {
      "price": "0.891",
      "side": "BUY",
      "size": "800"
    }
  ],
  "event_type": "price_change",
  "hash": "589157f43a921da5d7758b2c85bdb03c02660469",
  "market": "0xc40cbb2d7f5d2c43c624bd5b1d0b18cd3d0682b3937363ec7c1ad1d13bee107e",
  "timestamp": "1755746891701"
}
Price change update: [{'price': '0.891', 'side': 'BUY', 'size': '800'}]
[PRICE UPDATE] 104468181147316868388088006861839293041095272602974154655578369735976654024471
Best Bid: (0.9, 145.0) | Best Ask: (0.903, 449.0)
Mamdani YES | Best Bid: (0.837, 19.0) | Best Ask: (0.838, 7648.8)
Mamdani NO | Best Bid: (0.162, 7648.8) | Best Ask: (0.163, 19.0)
Cuomo NO | Best Bid: (0.9, 145.0) | Best Ask: (0.903, 449.0)
Cuomo YES | Best Bid: (0.097, 449.0) | Best Ask: (0.099, 19.44)

Received message — event_type: price_change
{
  "asset_id": "7268516239409850521789563


Received message — event_type: price_change
{
  "asset_id": "72685162394098505217895638060393901041260225434938300730127268362092284806692",
  "changes": [
    {
      "price": "0.101",
      "side": "SELL",
      "size": "45.04"
    }
  ],
  "event_type": "price_change",
  "hash": "22f3f666e1cbd6c17ad7680b93c1211d4b2e791f",
  "market": "0xc40cbb2d7f5d2c43c624bd5b1d0b18cd3d0682b3937363ec7c1ad1d13bee107e",
  "timestamp": "1755746897677"
}
Price change update: [{'price': '0.101', 'side': 'SELL', 'size': '45.04'}]
[PRICE UPDATE] 72685162394098505217895638060393901041260225434938300730127268362092284806692
Best Bid: (0.097, 449.0) | Best Ask: (0.099, 19.44)
Mamdani YES | Best Bid: (0.837, 19.0) | Best Ask: (0.838, 7648.8)
Mamdani NO | Best Bid: (0.162, 7648.8) | Best Ask: (0.163, 19.0)
Cuomo NO | Best Bid: (0.9, 153.34) | Best Ask: (0.903, 449.0)
Cuomo YES | Best Bid: (0.097, 449.0) | Best Ask: (0.099, 19.44)

Received message — event_type: price_change
{
  "asset_id": "104468181147316868

WebSocket error: 


### Kalshi

In [50]:
print_top_of_book()

Cuomo YES | Best Bid: (0.083, 12376.86) | Best Ask: (0.122, 12776.21)
Cuomo NO | Best Bid: (0.878, 12776.21) | Best Ask: (0.917, 12376.86)
Mamdani YES | Best Bid: (0.818, 32444.0) | Best Ask: (0.856, 30000.0)
Mamdani NO | Best Bid: (0.144, 30000.0) | Best Ask: (0.182, 32444.0)
